In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt

import gym

from gym import spaces

import players
import envs
from evaluation import generate_history, is_t4t

/home/peter/.local/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [2]:
import ray
from ray import tune
from ray.rllib.examples.env.multi_agent import MultiAgentCartPole
from ray.rllib.examples.models.shared_weights_model import \
    SharedWeightsModel1, SharedWeightsModel2, TF2SharedWeightsModel, \
    TorchSharedWeightsModel
from ray.rllib.models import ModelCatalog
# from ray.rllib.policy import PolicySpec
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import register_env

from ray.rllib.env.multi_agent_env import MultiAgentEnv


import ray
from ray.rllib.agents.ppo import PPOTrainer, PPOTorchPolicy, PPOTFPolicy
from ray.rllib.agents.ppo import DEFAULT_CONFIG as DEFAULT_CONFIG_PPO

from ray.rllib.agents.dqn import DQNTrainer, DQNTorchPolicy, DQNTFPolicy
from ray.rllib.agents.dqn import  DEFAULT_CONFIG as DEFAULT_CONFIG_DQN


from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

ray.init(ignore_reinit_error=True, log_to_driver=False)

2021-08-12 14:57:23,598	INFO services.py:1272 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.21',
 'raylet_ip_address': '192.168.1.21',
 'redis_address': '192.168.1.21:6379',
 'object_store_address': '/tmp/ray/session_2021-08-12_14-57-22_149583_113661/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-08-12_14-57-22_149583_113661/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-08-12_14-57-22_149583_113661',
 'metrics_export_port': 50589,
 'node_id': '8ee327bc8a28be62a453be14507154a8a803f94a3fd98defd6be4d33'}

In [3]:
register_env('two_agent_MG_env', lambda c: envs.TwoAgentMatrixGameEnv(c))

register_env('two_agent_t4t_MG_env', lambda c: envs.TwoAgentSeparateMatrixGameEnv(c))

register_env('MG_t4tTD_env', lambda c: envs.MatrixGameEnv(player2=players.TitForTatThenDefect()))
# register_env('MG_t4t_env', lambda c: envs.MatrixGameEnv(player2=players.TitForTat()))
register_env('MG_t4t_env', lambda c: envs.MatrixGameEnv())

In [90]:
trainer_config_dqn0 = DEFAULT_CONFIG_DQN.copy()
# trainer_config_dqn0['num_workers'] = 3
trainer_config_dqn0['n_step'] = 3
trainer_config_dqn0['noisy'] = True
trainer_config_dqn0['v_min'] = -10.
trainer_config_dqn0['v_max'] = 10.
trainer_config_dqn0['model']['fcnet_hiddens'] = [256,32,8]
trainer_config_dqn0['framework'] = 'torch'
# trainer_config_dqn0['framework'] = 'tf'

trainer_config_dqn0['env'] = 'MG_t4t_env'


In [91]:
# ray.init(ignore_reinit_error=True)
results = tune.run(
    "DQN",
    stop={"training_iteration":2},
    name='test_pretrain',
    config=trainer_config_dqn0,
#     config={
# #         "num_gpus": 0,
# #         "num_workers": 1,
# #         "lr": tune.grid_search([0.01, 0.001, 0.0001]),
#         "config":trainer_config_ppo
#     },
    verbose=1,
    checkpoint_at_end=True,
)

2021-08-12 12:14:05,017	INFO tune.py:549 -- Total run time: 8.56 seconds (8.06 seconds for the tuning loop).


In [92]:
agent0 = DQNTrainer(config=trainer_config_dqn0)

2021-08-12 12:14:07,953	WARNING util.py:53 -- Install gputil for GPU system monitoring.


In [93]:
agent0.restore(results.get_last_checkpoint())

2021-08-12 12:14:08,111	INFO trainable.py:377 -- Restored on 192.168.1.21 from checkpoint: /home/peter/ray_results/test_pretrain/DQN_MG_t4t_env_2f287_00000_0_2021-08-12_12-13-56/checkpoint_000002/checkpoint-2
2021-08-12 12:14:08,113	INFO trainable.py:385 -- Current state after restoring: {'_iteration': 2, '_timesteps_total': None, '_time_total': 5.200430631637573, '_episodes_total': 20}


In [94]:
results.get_last_checkpoint()

'/home/peter/ray_results/test_pretrain/DQN_MG_t4t_env_2f287_00000_0_2021-08-12_12-13-56/checkpoint_000002/checkpoint-2'

In [9]:
agent0.get_policy().get_weights()

{'_hidden_layers.0._model.0.weight': array([[ 0.04890376, -0.00558209,  0.01287127, ..., -0.07449174,
         -0.00684128, -0.03332094],
        [-0.07871381,  0.03723612, -0.01275583, ..., -0.03392028,
         -0.11493096, -0.03749057],
        [-0.01244541,  0.02239128, -0.00389348, ..., -0.01874868,
         -0.0041142 , -0.02274424],
        ...,
        [-0.05733058,  0.01309578, -0.10551378, ..., -0.0299821 ,
         -0.01164537, -0.0390831 ],
        [ 0.03285411, -0.00725919,  0.06943528, ..., -0.00349141,
         -0.02176144,  0.17167993],
        [-0.0195437 , -0.051215  ,  0.0139996 , ..., -0.01509678,
         -0.0730854 , -0.04187715]], dtype=float32),
 '_hidden_layers.0._model.0.bias': array([ 0.02052973,  0.00589235,  0.00153296, -0.02188277,  0.04248617,
        -0.01027183, -0.0207677 , -0.00118991, -0.02367345, -0.01769985,
        -0.01216752,  0.00134712,  0.03695444, -0.02303964,  0.02971013,
         0.00341944, -0.00409666,  0.03229688, -0.0179458 ,  0.018619

In [ ]:
def train(self, stop_criteria):
    """
    Train an RLlib PPO agent using tune until any of the configured stopping criteria is met.
    :param stop_criteria: Dict with stopping criteria.
        See https://docs.ray.io/en/latest/tune/api_docs/execution.html#tune-run
    :return: Return the path to the saved agent (checkpoint) and tune's ExperimentAnalysis object
        See https://docs.ray.io/en/latest/tune/api_docs/analysis.html#experimentanalysis-tune-experimentanalysis
    """
    analysis = ray.tune.run(ppo.PPOTrainer, config=self.config, local_dir=self.save_dir, stop=stop_criteria,
                            checkpoint_at_end=True)
    # list of lists: one list per checkpoint; each checkpoint list contains 1st the path, 2nd the metric value
    checkpoints = analysis.get_trial_checkpoints_paths(trial=analysis.get_best_trial('episode_reward_mean'),
                                                       metric='episode_reward_mean')
    # retriev the checkpoint path; we only have a single checkpoint, so take the first one
    checkpoint_path = checkpoints[0][0]
    return checkpoint_path, analysis

In [19]:
trainer_config_dqn0 = DEFAULT_CONFIG_DQN.copy()
# trainer_config_dqn0['num_workers'] = 3
trainer_config_dqn0['n_step'] = 3
trainer_config_dqn0['noisy'] = True
trainer_config_dqn0['v_min'] = -10.
trainer_config_dqn0['v_max'] = 10.
trainer_config_dqn0['model']['fcnet_hiddens'] = [256,32,8]
trainer_config_dqn0['framework'] = 'torch'
# trainer_config_dqn0['framework'] = 'tf'


In [20]:
trainer_config_dqn1 = DEFAULT_CONFIG_DQN.copy()
# trainer_config_dqn1['num_workers'] = 3
trainer_config_dqn1['n_step'] = 3
trainer_config_dqn1['noisy'] = True
trainer_config_dqn1['v_min'] = -10.
trainer_config_dqn1['v_max'] = 10.
trainer_config_dqn1['model']['fcnet_hiddens'] = [256,32,8]
trainer_config_dqn1['framework'] = 'torch'
# trainer_config_dqn1['framework'] = 'tf'


In [21]:
import copy

In [40]:
# set up stuff for pre-training with tit-for-tat

def env_creator(_):
    return envs.TwoAgentSeparateMatrixGameEnv()
single_env = envs.TwoAgentSeparateMatrixGameEnv()
env_name = 'two_agent_t4t_MG_env'
register_env(env_name, env_creator)



obs_space = single_env.observation_space
act_space = single_env.action_space
num_agents = single_env.num_agents
# DQNTorchPolicy0 = DQNTorchPolicy.copy()

def gen_policy(i):
#     return (PPOTorchPolicy, obs_space, act_space, {})
    if i == 0:
        conf = trainer_config_dqn0
#         conf['restore']='wtf_this better not work'#results.get_last_checkpoint()
#         return (DQNTorchPolicy0, obs_space, act_space, conf)
#         policy0 = copy.copy(agent0.get_policy())
#         policy0 = DQNTorchPolicy(obs_space, act_space, conf)
        policy0 = DQNTorchPolicy
#         policy0 = agent0.get_policy
#         policy0.set_weights(agent0.get_weights()['default_policy'])
#         policy0 = lambda c: policy0
#         return policy0
        return (policy0, obs_space, act_space, conf)

    elif i == 1:
        conf = trainer_config_dqn1
        return (DQNTorchPolicy, obs_space, act_space, conf)
    else:
        print('NO CONF!')
    
    return (DQNTorchPolicy, obs_space, act_space, conf)

policy_graphs = {}

# def restore_fn(agent_id):
# #     if agent_id == 0:
#     return results.get_last_checkpoint()
# #     else:
# #         return None
    
# restore_dict = {}
# for i in range(num_agents):
#     restore_dict['agent-' + str(i)] = restore_fn(i)

for i in range(num_agents):
    policy_graphs['agent-' + str(i)] = gen_policy(i)
def policy_mapping_fn(agent_id):
        return 'agent-' + str(agent_id)

In [41]:
# make config for pretraining with tit-for-tat

config={
    "log_level": "WARN",
#     "num_workers": 3,
#     "num_cpus_for_driver": 1,
#     "num_cpus_per_worker": 1,
#     "lr": 5e-3,
#     "model":{"fcnet_hiddens": [1024, 512,256,32,8]},
    "multiagent": {
        "policies": policy_graphs,
        "policy_mapping_fn": policy_mapping_fn,
    },
    "env": "two_agent_t4t_MG_env",
    'framework': 'torch',
#     'framework': 'tf',
#             "resume":True,
#         "restore": results.get_last_checkpoint()
    
}

In [44]:
# pretraining against tit-for-tat

exp_name = 'TA_TEST2'
exp_dict = {
        'name': exp_name,
        'run_or_experiment': 'DQN',
        "stop": {
            "training_iteration": 50
        },
        'checkpoint_freq': 0,
        'checkpoint_at_end': True,
        "config": config,
}
# ray.init()
MA_result=tune.run(**exp_dict)

Trial name,status,loc
DQN_two_agent_t4t_MG_env_30405_00000,PENDING,


Result for DQN_two_agent_t4t_MG_env_30405_00000:
  agent_timesteps_total: 2000
  custom_metrics:
    agent-0: {}
    agent-1: {}
  date: 2021-08-12_15-41-40
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 474.0
  episode_reward_mean: 455.6
  episode_reward_min: 420.0
  episodes_this_iter: 10
  episodes_total: 10
  experiment_id: fe9d180d9dcf4544b62ec7ccc2e5caca
  hostname: coolo-computer
  info:
    last_target_update_ts: 1000
    learner:
      agent-0:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 1.6481744050979614
        max_q: 0.3614499270915985
        mean_q: 0.04180954396724701
        mean_td_error: -3.8718888759613037
        min_q: -0.14508198201656342
      agent-1:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 5.041872978210449
        max_q: 0.6048080325126648
        mean_q: 0.27389317750930786
        mean_td_error: -3.382216215133667
        min_q: -0.013645298779010773
    num_a

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_t4t_MG_env_30405_00000,RUNNING,192.168.1.21:121225,1,3.60694,1000,455.6,474,420,100


Result for DQN_two_agent_t4t_MG_env_30405_00000:
  agent_timesteps_total: 4000
  custom_metrics:
    agent-0: {}
    agent-1: {}
  date: 2021-08-12_15-41-50
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 481.0
  episode_reward_mean: 457.3
  episode_reward_min: 416.0
  episodes_this_iter: 10
  episodes_total: 20
  experiment_id: fe9d180d9dcf4544b62ec7ccc2e5caca
  hostname: coolo-computer
  info:
    last_target_update_ts: 1504
    learner:
      agent-0:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 2.2518510818481445
        max_q: 11.82480525970459
        mean_q: 10.3193998336792
        mean_td_error: 1.7746717929840088
        min_q: 7.780777454376221
      agent-1:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 1.8816297054290771
        max_q: 12.973074913024902
        mean_q: 8.467057228088379
        mean_td_error: 1.6628128290176392
        min_q: 4.246359825134277
    num_agent_steps_sa

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_t4t_MG_env_30405_00000,RUNNING,192.168.1.21:121225,2,13.4908,2000,457.3,481,416,100


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_t4t_MG_env_30405_00000,RUNNING,192.168.1.21:121225,49,480.323,49000,585.62,600,530,100


Result for DQN_two_agent_t4t_MG_env_30405_00000:
  agent_timesteps_total: 100000
  custom_metrics:
    agent-0: {}
    agent-1: {}
  date: 2021-08-12_15-49-50
  done: true
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 600.0
  episode_reward_mean: 585.86
  episode_reward_min: 530.0
  episodes_this_iter: 10
  episodes_total: 500
  experiment_id: fe9d180d9dcf4544b62ec7ccc2e5caca
  hostname: coolo-computer
  info:
    last_target_update_ts: 49888
    learner:
      agent-0:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 1.5128663778305054
        max_q: 165.98611450195312
        mean_q: 107.05435180664062
        mean_td_error: 0.4205758273601532
        min_q: 18.293712615966797
      agent-1:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 0.49860724806785583
        max_q: 171.54190063476562
        mean_q: 117.03024291992188
        mean_td_error: 0.6482353806495667
        min_q: 2.658019781112671
    num_agen

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_t4t_MG_env_30405_00000,RUNNING,192.168.1.21:121225,50,491.639,50000,585.86,600,530,100


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_t4t_MG_env_30405_00000,TERMINATED,,50,491.639,50000,585.86,600,530,100


2021-08-12 15:49:50,608	INFO tune.py:549 -- Total run time: 496.91 seconds (496.58 seconds for the tuning loop).


In [45]:
# setting up stuff for 1 v 1 training

def env_creator(_):
    return envs.TwoAgentMatrixGameEnv()
single_env = envs.TwoAgentMatrixGameEnv()
env_name = "two_agent_MG_env"
register_env(env_name, env_creator)



obs_space = single_env.observation_space
act_space = single_env.action_space
num_agents = single_env.num_agents
# DQNTorchPolicy0 = DQNTorchPolicy.copy()

def gen_policy(i):
#     return (PPOTorchPolicy, obs_space, act_space, {})
    if i == 0:
        conf = trainer_config_dqn0
#         conf['restore']='wtf_this better not work'#results.get_last_checkpoint()
#         return (DQNTorchPolicy0, obs_space, act_space, conf)
#         policy0 = copy.copy(agent0.get_policy())
#         policy0 = DQNTorchPolicy(obs_space, act_space, conf)
        policy0 = DQNTorchPolicy
#         policy0 = agent0.get_policy
#         policy0.set_weights(agent0.get_weights()['default_policy'])
#         policy0 = lambda c: policy0
#         return policy0
        return (policy0, obs_space, act_space, conf)

    elif i == 1:
        conf = trainer_config_dqn1
        return (DQNTorchPolicy, obs_space, act_space, conf)
    else:
        print('NO CONF!')
    
    return (DQNTorchPolicy, obs_space, act_space, conf)

policy_graphs = {}

# def restore_fn(agent_id):
# #     if agent_id == 0:
#     return results.get_last_checkpoint()
# #     else:
# #         return None
    
# restore_dict = {}
# for i in range(num_agents):
#     restore_dict['agent-' + str(i)] = restore_fn(i)

for i in range(num_agents):
    policy_graphs['agent-' + str(i)] = gen_policy(i)
def policy_mapping_fn(agent_id):
        return 'agent-' + str(agent_id)

In [46]:
# making config for 1-v-1 training

config={
    "log_level": "WARN",
#     "num_workers": 3,
#     "num_cpus_for_driver": 1,
#     "num_cpus_per_worker": 1,
#     "lr": 5e-3,
#     "model":{"fcnet_hiddens": [1024, 512,256,32,8]},
    "multiagent": {
        "policies": policy_graphs,
        "policy_mapping_fn": policy_mapping_fn,
    },
    "env": "two_agent_MG_env",
    'framework': 'torch',
#     'framework': 'tf',
#             "resume":True,
#         "restore": results.get_last_checkpoint()
    
}

In [53]:
# doing 1-v-1 training
# with pretrained agents
checkpoint = MA_result.get_last_checkpoint()
checkpoint = '/home/peter/ray_results/TA_TEST2/DQN_two_agent_t4t_MG_env_30405_00000_0_2021-08-12_15-41-33/checkpoint_000050/checkpoint-50'
exp_name = 'TA_TEST2'
exp_dict = {
        'name': exp_name,
        'run_or_experiment': 'DQN',
        "stop": {
            "training_iteration": 100
        },
        'checkpoint_freq': 0,
        'checkpoint_at_end': True,
        "config": config,
        "restore": checkpoint

}
# ray.init()
MA_result=tune.run(**exp_dict)

Trial name,status,loc
DQN_two_agent_MG_env_06046_00000,PENDING,


Result for DQN_two_agent_MG_env_06046_00000:
  agent_timesteps_total: 102000
  custom_metrics:
    agent-0: {}
    agent-1: {}
  date: 2021-08-12_15-54-49
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 476.0
  episode_reward_mean: 455.8
  episode_reward_min: 438.0
  episodes_this_iter: 10
  episodes_total: 510
  experiment_id: fe9d180d9dcf4544b62ec7ccc2e5caca
  hostname: coolo-computer
  info:
    last_target_update_ts: 51000
    learner:
      agent-0:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 27.034523010253906
        max_q: 155.60800170898438
        mean_q: 99.80780029296875
        mean_td_error: 1.153451919555664
        min_q: 5.13901424407959
      agent-1:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 16.904460906982422
        max_q: 173.2588653564453
        mean_q: 112.38602447509766
        mean_td_error: 1.7448171377182007
        min_q: 3.8681387901306152
    num_agent_steps_s

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_MG_env_06046_00000,RUNNING,192.168.1.21:127629,51,495.512,51000,455.8,476,438,100


Result for DQN_two_agent_MG_env_06046_00000:
  agent_timesteps_total: 104000
  custom_metrics:
    agent-0: {}
    agent-1: {}
  date: 2021-08-12_15-54-59
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 485.0
  episode_reward_mean: 395.2
  episode_reward_min: 224.0
  episodes_this_iter: 10
  episodes_total: 520
  experiment_id: fe9d180d9dcf4544b62ec7ccc2e5caca
  hostname: coolo-computer
  info:
    last_target_update_ts: 51504
    learner:
      agent-0:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 4.2104597091674805
        max_q: 166.86334228515625
        mean_q: 108.39470672607422
        mean_td_error: 3.252502679824829
        min_q: 16.601627349853516
      agent-1:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 2.75736927986145
        max_q: 161.55484008789062
        mean_q: 102.17491149902344
        mean_td_error: 1.1177082061767578
        min_q: 24.189285278320312
    num_agent_steps

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_MG_env_06046_00000,RUNNING,192.168.1.21:127629,52,505.096,52000,395.2,485,224,100


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_MG_env_06046_00000,RUNNING,192.168.1.21:127629,99,971.108,99000,215.14,238,203,100


Result for DQN_two_agent_MG_env_06046_00000:
  agent_timesteps_total: 200000
  custom_metrics:
    agent-0: {}
    agent-1: {}
  date: 2021-08-12_16-02-55
  done: true
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 238.0
  episode_reward_mean: 214.89
  episode_reward_min: 203.0
  episodes_this_iter: 10
  episodes_total: 1000
  experiment_id: fe9d180d9dcf4544b62ec7ccc2e5caca
  hostname: coolo-computer
  info:
    last_target_update_ts: 99888
    learner:
      agent-0:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 1.8093993663787842
        max_q: 149.7140655517578
        mean_q: 33.36493682861328
        mean_td_error: 2.048804759979248
        min_q: 0.9183366298675537
      agent-1:
        allreduce_latency: 0.0
        cur_lr: 0.0005
        grad_gnorm: 0.7457124590873718
        max_q: 127.22663879394531
        mean_q: 35.550045013427734
        mean_td_error: 2.2260630130767822
        min_q: 0.9635365009307861
    num_agent_step

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_MG_env_06046_00000,RUNNING,192.168.1.21:127629,100,980.393,100000,214.89,238,203,100


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_MG_env_06046_00000,TERMINATED,,100,980.393,100000,214.89,238,203,100


2021-08-12 16:02:56,339	INFO tune.py:549 -- Total run time: 494.50 seconds (494.15 seconds for the tuning loop).


In [58]:
MA_result.results

{'06046_00000': {'episode_reward_max': 238.0,
  'episode_reward_min': 203.0,
  'episode_reward_mean': 214.89,
  'episode_len_mean': 100.0,
  'episode_media': {},
  'episodes_this_iter': 10,
  'policy_reward_min': {'agent-0': 99.0, 'agent-1': 96.0},
  'policy_reward_max': {'agent-0': 121.0, 'agent-1': 119.0},
  'policy_reward_mean': {'agent-0': 110.42, 'agent-1': 104.47},
  'custom_metrics': {'agent-0': {}, 'agent-1': {}},
  'hist_stats': {'episode_reward': [216.0,
    220.0,
    216.0,
    215.0,
    213.0,
    219.0,
    214.0,
    219.0,
    228.0,
    207.0,
    217.0,
    211.0,
    220.0,
    220.0,
    220.0,
    220.0,
    216.0,
    216.0,
    229.0,
    213.0,
    206.0,
    212.0,
    209.0,
    212.0,
    222.0,
    209.0,
    206.0,
    210.0,
    213.0,
    212.0,
    217.0,
    214.0,
    203.0,
    225.0,
    206.0,
    217.0,
    215.0,
    211.0,
    238.0,
    220.0,
    220.0,
    226.0,
    213.0,
    213.0,
    216.0,
    214.0,
    214.0,
    210.0,
    216.0,
   